In [1]:
import os
import pandas as pd
import numpy as np

import torch
from torch import nn
from torchmetrics import Accuracy, Recall, Precision, AUROC

from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_cosine_schedule_with_warmup
import datasets

from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

/storage/home/hcoda1/1/yyu496/.conda/envs/BrainBrige/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('~/p-cwiese7-0/data/final_jargon_dataset.csv')

In [3]:
df = pd.read_csv('~/p-cwiese7-0/data/final_jargon_dataset.csv')
df['Label'] = df['discipline'].apply(lambda x: 0 if x == 'No Discipline' else 1)

In [4]:
df

,jargon,translation,discipline,sentence,Label
0,Cantilever,A long projecting beam or girder fixed at only...,Architecture,The innovative use of cantilevered structures ...,1
1,Facade,"The face of a building, especially the princip...",Architecture,The design of the building's facade not only e...,1
2,Fenestration,The design and placement of windows in a build...,Architecture,"In architectural design, fenestration plays a ...",1
3,Pediment,The triangular upper part of the front of a bu...,Architecture,"The pediment, typically characterized by its t...",1
4,Quoin,"The outside corner of a wall or building, ofte...",Architecture,The structural integrity of the building was s...,1
...,...,...,...,...,...
5431,idea ripple,No translation,No Discipline,We need to consider the idea ripple that our p...,0
5432,thought ripple,No translation,No Discipline,The recent discussion sparked a thought ripple...,0
5433,thoughtwave,No translation,No Discipline,We need to ensure that our thoughtwaves align ...,0
5434,thought ripple,No translation,No Discipline,The recent discussion on sustainability initia...,0


In [5]:
class textDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, tokenizer):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        tokenized_texts_and_mask = self.tokenizer(
            self.dataframe.sentence.iloc[idx],
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        encoded_Label = torch.tensor(self.dataframe.Encoded_Label.iloc[idx], dtype=torch.long)
        
        return tokenized_texts_and_mask['input_ids'].squeeze(0), tokenized_texts_and_mask['attention_mask'].squeeze(0), encoded_Label

In [6]:
class get_torch_datasets_and_dataloaders():
    def __init__(self, dataframe, tokenizer_type, encode, torch_costum_dataset):
        self.dataframe = dataframe
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_type)
        self.encode = encode
        self.torch_costum_dataset = torch_costum_dataset
        
        if self.encode:
            self.dataframe['Encoded_Label'] = LabelEncoder().fit_transform(self.dataframe.Label)
        else:
            self.dataframe.rename(columns={'Label' : 'Encoded_Label'}, inplace=True)
        
        
    def get_torch_dataset(self, split):
        if split:
            df_train, temp = train_test_split(self.dataframe, test_size=0.8, random_state=42)
            df_valid, df_test = train_test_split(temp, test_size=0.5, random_state=42)
            
            return {
                'train' : self.torch_costum_dataset(df_train, self.tokenizer),
                'valid' : self.torch_costum_dataset(df_valid, self.tokenizer),
                'test' : self.torch_costum_dataset(df_test, self.tokenizer)
            }
        else:
            return self.torch_costum_dataset(self.dataframe, self.tokenizer)
        
    def get_torch_dataloader(self, split, batch_size, pin_memory, shuffle=False):
        self.num_cpu = len(os.sched_getaffinity(0))
        
        if split:
            torch_datasets = self.get_torch_dataset(split)
            
            return {
                'train' : torch.utils.data.DataLoader(torch_datasets['train'], shuffle=shuffle, batch_size=batch_size, num_workers=self.num_cpu, pin_memory=pin_memory, persistent_workers=self.num_cpu),
                'valid' : torch.utils.data.DataLoader(torch_datasets['valid'], batch_size=batch_size, num_workers=self.num_cpu, pin_memory=pin_memory, persistent_workers=self.num_cpu),
                'test' : torch.utils.data.DataLoader(torch_datasets['test'], batch_size=batch_size, num_workers=self.num_cpu, pin_memory=pin_memory, persistent_workers=self.num_cpu)
            }
        else:
            torch_datasets = self.get_torch_dataset(split)
            return torch.utils.data.DataLoader(self.torch_datasets, shuffle=shuffle, batch_size=batch_size, num_workers=self.num_cpu, pin_memory=pin_memory, persistent_workers=persistent_workers)
        
        

In [7]:
class EarlyStop():
    def __init__(self, model_type, patience=5, verbose=False, delta=0, ):
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.counter = 0
        self.best_score = 0
        self.early_stop = False
        self.model_type = model_type

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score == 0:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'Patience Counte: {self.counter}/{self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            if self.verbose:
                print(f'Currently, the min val_loss: {val_loss}')
            self.save_checkpoint(model)

    def save_checkpoint(self, model):
        torch.save(model.state_dict(), f'/storage/coda1/p-cwiese7/0/yyu496/Model/checkpoint_{self.model_type}.pt')

In [8]:
def train_one_epoch(model, dataloader, loss_func, optimizer, scheduler, scaler, metrics, device, epoch, num_epochs):
    model.train()
    train_loss = 0
    total_outputs, total_preds, total_labels = [], [], []

    progress_bar = tqdm(dataloader, total=len(dataloader), desc=f'Epoch: {epoch}/{num_epochs}')
    with torch.amp.autocast(device_type=device):
        for batch in progress_bar:

            input_ids, attention_mask, labels = batch
            
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)
            
            outputs = model(
                input_ids=input_ids, 
                attention_mask=attention_mask
            )
            loss = loss_func(outputs.logits, labels)

            preds = torch.argmax(outputs.logits, dim=1)
            train_loss += loss.item()

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            optimizer.zero_grad()

            total_outputs.append(outputs.logits)
            total_preds.append(preds)
            total_labels.append(labels)

        total_outputs = torch.cat(total_outputs)
        #total_outputs_prob = torch.softmax(total_outputs, dim=1)[:, 1] # Only for Binary
        total_preds = torch.cat(total_preds)
        total_labels = torch.cat(total_labels)

        current_lr = scheduler.optimizer.param_groups[0]['lr']
        training_loss = train_loss / len(dataloader)
        training_accuracy = metrics['Accuracy'](total_preds, total_labels)
        training_recall = metrics['Recall'](total_preds, total_labels)
        training_precision = metrics['Precision'](total_preds, total_labels)
        #training_auroc = metrics['Auroc'](total_outputs_prob, total_labels) # For Binary
        training_auroc = metrics['Auroc'](total_outputs, total_labels) # For Multi Class

    return {
        'Training_Loss' : training_loss,
        'Training_LR' : current_lr,
        'Training_Accuracy' : training_accuracy,
        'Training_Recall' : training_recall,
        'Training_Precision' : training_precision,
        'Training_Auroc' : training_auroc
    }


In [9]:
def validation(model, dataloader, loss_func, metrics, device):
    model.eval()
    validation_loss = 0
    validation_outputs, validation_preds, validation_labels = [], [], []

    progress_bar = tqdm(dataloader, total=len(dataloader), desc='Validation: ')

    with torch.no_grad():
        with torch.amp.autocast(device_type=device):
            for batch in progress_bar:
                
                input_ids, attention_mask, labels = batch
            
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                labels = labels.to(device)
                
                outputs = model(
                    input_ids = input_ids, 
                    attention_mask = attention_mask)
                
                loss = loss_func(outputs.logits, labels)
                
                preds = torch.argmax(outputs.logits, dim=1)
                validation_loss += loss.item()

                validation_outputs.append(outputs.logits)
                validation_preds.append(preds)
                validation_labels.append(labels)

            validation_outputs = torch.cat(validation_outputs)
            #validation_outputs_prob = torch.softmax(validation_outputs, dim=1)[:, 1] # Only for Binary
            validation_preds = torch.cat(validation_preds)
            validation_labels = torch.cat(validation_labels)

            validation_loss = validation_loss / len(dataloader)
            validation_accuracy = metrics['Accuracy'](validation_preds, validation_labels)
            validation_recall = metrics['Recall'](validation_preds, validation_labels)
            validation_precision = metrics['Precision'](validation_preds, validation_labels)
            #validation_auroc = metrics['Auroc'](validation_outputs_prob, validation_labels) # For Binary
            validation_auroc = metrics['Auroc'](validation_outputs, validation_labels) # For Multi Class
     
    return {
        'Validation_Loss' : validation_loss,
        'Validation_Accuracy' : validation_accuracy,
        'Validation_Recall' : validation_recall,
        'Validation_Precision' : validation_precision,
        'Validation_Auroc' : validation_auroc
    }

In [14]:
def testing(model, dataloader, loss_func, metrics, device):
    model.eval()
    testing_loss = 0
    testing_outputs, testing_preds, testing_labels = [], [], []
    
    progress_bar = tqdm(dataloader, total=len(dataloader), desc='Testing: ')
    
    with torch.no_grad():
        with torch.amp.autocast(device_type=device):
            for batch in progress_bar:
                
                input_ids, attention_mask, labels = batch
            
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                labels = labels.to(device)
                
                outputs = model(
                input_ids = input_ids,
                attention_mask = attention_mask)
                
                loss = loss_func(outputs.logits, labels)
                
                preds = torch.argmax(outputs.logits, dim=1)
                testing_loss += loss.item()
                
                testing_outputs.append(outputs.logits)
                testing_preds.append(preds)
                testing_labels.append(labels)
            
            testing_outputs = torch.cat(testing_outputs)
            #testing_outputs_prob = torch.softmax(testing_outputs, dim=1)[:, 1] Only for Binary
            testing_preds = torch.cat(testing_preds)
            testing_labels = torch.cat(testing_labels)
            
            testing_loss = testing_loss / len(dataloader)
            testing_accuracy = metrics['Accuracy'](testing_preds, testing_labels)
            testing_recall = metrics['Recall'](testing_preds, testing_labels)
            testing_precision = metrics['Precision'](testing_preds, testing_labels)
            #testing_auroc = metrics['Auroc'](testing_outputs_prob, testing_labels) # For Binary
            testing_auroc = metrics['Auroc'](testing_outputs, testing_labels) # For Multi Class
            
    return {
        'Testing_Loss' : testing_loss,
        'Testing_Accuracy' : testing_accuracy,
        'Testing_Recall' : testing_recall,
        'Testing_Precision' : testing_precision,
        'Testing_Auroc' : testing_auroc
    }

In [11]:
def main_binary(model, num_epochs=32, patience=3, batch_size=64, state=None):

    # -------
    # Device
    # -------
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f'Using Device: {device}')

    # ------
    # Model
    # ------
    model = model
    model.to(device)

    torch_dataloader = get_torch_datasets_and_dataloaders(
        dataframe=df, tokenizer_type='bert-base-uncased', encode=False, torch_costum_dataset=textDataset).get_torch_dataloader(split=True, shuffle=True, batch_size=32, pin_memory=True)
    training_data_final = torch_dataloader['train']
    validation_data_final = torch_dataloader['valid']
    testing_data_final = torch_dataloader['test']
    
    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.0005)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5)
    scaler = torch.amp.GradScaler()
    early_stopping = EarlyStop(model_type='binaary', patience=5, verbose=True)

    accuracy = Accuracy(task='binary').to(device)
    recall = Recall(task='binary').to(device)
    precision = Precision(task='binary').to(device)
    auroc = AUROC(task='binary').to(device)
    metrics = {
        'Accuracy' : accuracy,
        'Recall' : recall,
        'Precision' : precision,
        'Auroc' : auroc
    }

    if state == 'train':
        val_loss = 0
        for epoch in range(1, num_epochs+1):
            training_metrics = train_one_epoch(model=model, dataloader=training_data_final, loss_func=loss_func, optimizer=optimizer, scheduler=scheduler, scaler=scaler, metrics=metrics, device=device, epoch=epoch, num_epochs=num_epochs)

            scheduler.step(val_loss)  
                
            validation_metrics = validation(model=model, dataloader=validation_data_final, loss_func=loss_func, metrics=metrics, device=device)
            val_loss = validation_metrics['Validation_Loss']

            print(f"""Training Metrics: Training Loss: {training_metrics['Training_Loss']:.7f} || Training Accuracy: {training_metrics['Training_Accuracy']}
                                        Training Precision: {training_metrics['Training_Precision']} || Training Recall: {training_metrics['Training_Recall']}
                                        Training AUROC: {training_metrics['Training_Auroc']} || Training Learning Rate: {training_metrics['Training_LR']}""")
            print('-'*70, '\n')
            print(f"""Validation Metrics: Validation Loss: {validation_metrics['Validation_Loss']:.7f} || Validation Accuracy: {validation_metrics['Validation_Accuracy']}
                                          Validation Precision: {validation_metrics['Validation_Precision']} || Validation Recall: {validation_metrics['Validation_Recall']}
                                          Validation AUROC: {validation_metrics['Validation_Auroc']}""")
            # Check for early stopping
            early_stopping(val_loss, model)
            if early_stopping.early_stop == True:
                break
    elif state == 'eval':
        if os.path.exists('/storage/coda1/p-cwiese7/0/yyu496/Model/checkpoint_binary.pt'):
            model.load_state_dict(torch.load('/storage/coda1/p-cwiese7/0/yyu496/Model/checkpoint_binary.pt'))
            testing_metrics = testing(model=model, dataloader=testing_data_final, loss_func=loss_func, metrics=metrics, device=device)
            print(f"""Testing Metrics: Testing Loss: {testing_metrics['Testing_Loss']:.7f} || Testing Accuracy: {testing_metrics['Testing_Accuracy']}
                                       Testing Precision: {testing_metrics['Testing_Precision']} || Testing Recall: {testing_metrics['Testing_Recall']}
                                       Testing AUROC: {testing_metrics['Testing_Auroc']}""")
        else:
            raise ValueError('Please train the model first!')
    else:
        raise ValueError('Please choose the state between train and eval!')
    

In [12]:
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
%%time 
main_binary(model, state='train', num_epochs=64)

Using Device: cuda


Validation: 100%|██████████| 68/68 [00:04<00:00, 13.71it/s]


Training Metrics: Training Loss: 0.6902961 || Training Accuracy: 0.5298988223075867
                                        Training Precision: 0.8560371398925781 || Training Recall: 0.5695159435272217
                                        Training AUROC: 0.355618953704834 || Training Learning Rate: 1e-05
---------------------------------------------------------------------- 

Validation Metrics: Validation Loss: 0.3984390 || Validation Accuracy: 0.9020239114761353
                                          Validation Precision: 0.9020239114761353 || Validation Recall: 1.0
                                          Validation AUROC: 0.942529559135437


Validation: 100%|██████████| 68/68 [00:05<00:00, 13.39it/s]


Training Metrics: Training Loss: 0.4110351 || Training Accuracy: 0.893284261226654
                                        Training Precision: 0.893284261226654 || Training Recall: 1.0
                                        Training AUROC: 0.6448959708213806 || Training Learning Rate: 1e-05
---------------------------------------------------------------------- 

Validation Metrics: Validation Loss: 0.3109497 || Validation Accuracy: 0.9020239114761353
                                          Validation Precision: 0.9020239114761353 || Validation Recall: 1.0
                                          Validation AUROC: 0.9795998334884644
Currently, the min val_loss: 0.31094974717673135


...

Training Metrics: Training Loss: 0.0059374 || Training Accuracy: 1.0
                                        Training Precision: 1.0 || Training Recall: 1.0
                                        Training AUROC: 1.0 || Training Learning Rate: 1.953125e-08
---------------------------------------------------------------------- 

Validation Metrics: Validation Loss: 0.0047929 || Validation Accuracy: 1.0
                                          Validation Precision: 1.0 || Validation Recall: 1.0
                                          Validation AUROC: 1.0
Currently, the min val_loss: 0.004792906665791045


Validation: 100%|██████████| 68/68 [00:04<00:00, 13.97it/s]


Training Metrics: Training Loss: 0.0059861 || Training Accuracy: 1.0
                                        Training Precision: 1.0 || Training Recall: 1.0
                                        Training AUROC: 1.0 || Training Learning Rate: 1.953125e-08
---------------------------------------------------------------------- 

Validation Metrics: Validation Loss: 0.0047780 || Validation Accuracy: 1.0
                                          Validation Precision: 1.0 || Validation Recall: 1.0
                                          Validation AUROC: 1.0
Currently, the min val_loss: 0.004777999834486228


Validation: 100%|██████████| 68/68 [00:04<00:00, 14.02it/s]


Training Metrics: Training Loss: 0.0059092 || Training Accuracy: 1.0
                                        Training Precision: 1.0 || Training Recall: 1.0
                                        Training AUROC: 1.0 || Training Learning Rate: 1.953125e-08
---------------------------------------------------------------------- 

Validation Metrics: Validation Loss: 0.0047625 || Validation Accuracy: 1.0
                                          Validation Precision: 1.0 || Validation Recall: 1.0
                                          Validation AUROC: 1.0
Currently, the min val_loss: 0.004762499709613621
CPU times: user 15min 4s, sys: 29.4 s, total: 15min 33s
Wall time: 15min 52s


In [13]:
main_binary(model, state='eval')

Using Device: cuda


/tmp/ipykernel_2461889/336821419.py:61: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/storage/coda1/p-cwiese7/0/yyu496/Model/checkpoint_bi

Testing Metrics: Testing Loss: 0.0048430 || Testing Accuracy: 1.0
                                       Testing Precision: 1.0 || Testing Recall: 1.0
                                       Testing AUROC: 1.0


In [15]:
def main_multiclass(model, num_epochs=32, patience=3, batch_size=64, state=None):

    # -------
    # Device
    # -------
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f'Using Device: {device}')

    # ------
    # Model
    # ------
    model = model
    model.to(device)

    df = pd.read_csv('~/p-cwiese7-0/data/final_jargon_dataset.csv')
    df['Label'] = df['discipline'].apply(lambda x: 0 if x == 'No Discipline' else 1)
    df.rename(columns={'Label':'Contains_Jargen'}, inplace=True)
    df.rename(columns={'discipline':'Label'}, inplace=True)
    
    torch_dataloader = get_torch_datasets_and_dataloaders(
        dataframe=df, tokenizer_type='bert-base-uncased', encode=True, torch_costum_dataset=textDataset).get_torch_dataloader(split=True, shuffle=True, batch_size=32, pin_memory=True)
    training_data_final = torch_dataloader['train']
    validation_data_final = torch_dataloader['valid']
    testing_data_final = torch_dataloader['test']
    
    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.00005)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.3)
    scaler = torch.amp.GradScaler()
    early_stopping = EarlyStop(model_type='multiclass', patience=5, verbose=True)

    accuracy = Accuracy(task='multiclass', num_classes=4, average='none').to(device)
    recall = Recall(task='multiclass', num_classes=4, average='none').to(device)
    precision = Precision(task='multiclass', num_classes=4, average='none').to(device)
    auroc = AUROC(task='multiclass', num_classes=4, average='none').to(device)
    metrics = {
        'Accuracy' : accuracy,
        'Recall' : recall,
        'Precision' : precision,
        'Auroc' : auroc
    }

    if state == 'train':
        val_loss = 0
        for epoch in range(1, num_epochs+1):
            training_metrics = train_one_epoch(model=model, dataloader=training_data_final, loss_func=loss_func, optimizer=optimizer, scheduler=scheduler, scaler=scaler, metrics=metrics, device=device, epoch=epoch, num_epochs=num_epochs)

            scheduler.step(val_loss)  
                
            validation_metrics = validation(model=model, dataloader=validation_data_final, loss_func=loss_func, metrics=metrics, device=device)
            val_loss = validation_metrics['Validation_Loss']

            print(f"""Training Metrics: Training Loss: {training_metrics['Training_Loss']:.7f} || Training Accuracy: {training_metrics['Training_Accuracy']}
                                        Training Precision: {training_metrics['Training_Precision']} || Training Recall: {training_metrics['Training_Recall']}
                                        Training AUROC: {training_metrics['Training_Auroc']} || Training Learning Rate: {training_metrics['Training_LR']}""")
            print('-'*70, '\n')
            print(f"""Validation Metrics: Validation Loss: {validation_metrics['Validation_Loss']:.7f} || Validation Accuracy: {validation_metrics['Validation_Accuracy']}
                                          Validation Precision: {validation_metrics['Validation_Precision']} || Validation Recall: {validation_metrics['Validation_Recall']}
                                          Validation AUROC: {validation_metrics['Validation_Auroc']}""")
            # Check for early stopping
            early_stopping(val_loss, model)
            if early_stopping.early_stop == True:
                break
    elif state == 'eval':
        if os.path.exists('/storage/coda1/p-cwiese7/0/yyu496/Model/checkpoint_multiclass.pt'):
            model.load_state_dict(torch.load('/storage/coda1/p-cwiese7/0/yyu496/Model/checkpoint_multiclass.pt'))
            testing_metrics = testing(model=model, dataloader=testing_data_final, loss_func=loss_func, metrics=metrics, device=device)
            print(f"""Testing Metrics: Testing Loss: {testing_metrics['Testing_Loss']:.7f} || Testing Accuracy: {testing_metrics['Testing_Accuracy']}
                                       Testing Precision: {testing_metrics['Testing_Precision']} || Testing Recall: {testing_metrics['Testing_Recall']}
                                       Testing AUROC: {testing_metrics['Testing_Auroc']}""")
        else:
            raise ValueError('Please train the model first!')
    else:
        raise ValueError('Please choose the state between train and eval!')
    

In [16]:
model_multiclass = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
main_multiclass(model_multiclass, state='train', num_epochs=64)

Using Device: cuda


Validation: 100%|██████████| 68/68 [00:05<00:00, 13.50it/s]


Training Metrics: Training Loss: 1.5459923 || Training Accuracy: tensor([0.0000, 0.0560, 0.8017, 0.1320], device='cuda:0')
                                        Training Precision: tensor([0.0000, 0.5758, 0.1045, 0.2484], device='cuda:0') || Training Recall: tensor([0.0000, 0.0560, 0.8017, 0.1320], device='cuda:0')
                                        Training AUROC: tensor([0.3286, 0.5905, 0.3691, 0.4619], device='cuda:0') || Training Learning Rate: 1e-05
---------------------------------------------------------------------- 

Validation Metrics: Validation Loss: 1.3633529 || Validation Accuracy: tensor([0.6080, 0.0817, 0.0000, 0.0000], device='cuda:0')
                                          Validation Precision: tensor([0.2561, 0.0896, 0.0000, 0.0000], device='cuda:0') || Validation Recall: tensor([0.6080, 0.0817, 0.0000, 0.0000], device='cuda:0')
                                          Validation AUROC: tensor([0.3356, 0.2749, 0.0540, 0.4911], device='cuda:0')


Validation: 100%|██████████| 68/68 [00:04<00:00, 14.09it/s]


Training Metrics: Training Loss: 1.3619321 || Training Accuracy: tensor([0.3830, 0.4631, 0.0000, 0.0033], device='cuda:0')
                                        Training Precision: tensor([0.2819, 0.2465, 0.0000, 0.3333], device='cuda:0') || Training Recall: tensor([0.3830, 0.4631, 0.0000, 0.0033], device='cuda:0')
                                        Training AUROC: tensor([0.4679, 0.4168, 0.4495, 0.5065], device='cuda:0') || Training Learning Rate: 1e-05
---------------------------------------------------------------------- 

Validation Metrics: Validation Loss: 1.3005646 || Validation Accuracy: tensor([0.0000, 0.9824, 0.0000, 0.8079], device='cuda:0')
                                          Validation Precision: tensor([0.0000, 0.4964, 0.0000, 0.5687], device='cuda:0') || Validation Recall: tensor([0.0000, 0.9824, 0.0000, 0.8079], device='cuda:0')
                                          Validation AUROC: tensor([0.8600, 0.9845, 0.9730, 0.7493], device='cuda:0')
Currently, t

...

Training Metrics: Training Loss: 0.1591510 || Training Accuracy: tensor([0.9939, 1.0000, 0.9914, 1.0000], device='cuda:0')
                                        Training Precision: tensor([1.0000, 0.9912, 1.0000, 1.0000], device='cuda:0') || Training Recall: tensor([0.9939, 1.0000, 0.9914, 1.0000], device='cuda:0')
                                        Training AUROC: tensor([0.9991, 0.9994, 1.0000, 1.0000], device='cuda:0') || Training Learning Rate: 7.289999999999999e-09
---------------------------------------------------------------------- 

Validation Metrics: Validation Loss: 0.1468225 || Validation Accuracy: tensor([0.9941, 1.0000, 0.9765, 1.0000], device='cuda:0')
                                          Validation Precision: tensor([1.0000, 0.9858, 1.0000, 1.0000], device='cuda:0') || Validation Recall: tensor([0.9941, 1.0000, 0.9765, 1.0000], device='cuda:0')
                                          Validation AUROC: tensor([0.9996, 0.9999, 1.0000, 1.0000], device='cuda:

Validation: 100%|██████████| 68/68 [00:04<00:00, 14.06it/s]


Training Metrics: Training Loss: 0.1591999 || Training Accuracy: tensor([0.9939, 1.0000, 1.0000, 1.0000], device='cuda:0')
                                        Training Precision: tensor([1.0000, 0.9941, 1.0000, 1.0000], device='cuda:0') || Training Recall: tensor([0.9939, 1.0000, 1.0000, 1.0000], device='cuda:0')
                                        Training AUROC: tensor([0.9988, 0.9991, 1.0000, 1.0000], device='cuda:0') || Training Learning Rate: 7.289999999999999e-09
---------------------------------------------------------------------- 

Validation Metrics: Validation Loss: 0.1465999 || Validation Accuracy: tensor([0.9941, 1.0000, 0.9765, 1.0000], device='cuda:0')
                                          Validation Precision: tensor([1.0000, 0.9858, 1.0000, 1.0000], device='cuda:0') || Validation Recall: tensor([0.9941, 1.0000, 0.9765, 1.0000], device='cuda:0')
                                          Validation AUROC: tensor([0.9996, 0.9999, 1.0000, 1.0000], device='cuda:

Validation: 100%|██████████| 68/68 [00:04<00:00, 13.92it/s]


Training Metrics: Training Loss: 0.1609234 || Training Accuracy: tensor([0.9909, 1.0000, 1.0000, 1.0000], device='cuda:0')
                                        Training Precision: tensor([1.0000, 0.9912, 1.0000, 1.0000], device='cuda:0') || Training Recall: tensor([0.9909, 1.0000, 1.0000, 1.0000], device='cuda:0')
                                        Training AUROC: tensor([0.9997, 0.9998, 1.0000, 1.0000], device='cuda:0') || Training Learning Rate: 7.289999999999999e-09
---------------------------------------------------------------------- 

Validation Metrics: Validation Loss: 0.1463613 || Validation Accuracy: tensor([0.9941, 1.0000, 0.9765, 1.0000], device='cuda:0')
                                          Validation Precision: tensor([1.0000, 0.9858, 1.0000, 1.0000], device='cuda:0') || Validation Recall: tensor([0.9941, 1.0000, 0.9765, 1.0000], device='cuda:0')
                                          Validation AUROC: tensor([0.9996, 0.9999, 1.0000, 1.0000], device='cuda:

In [17]:
main_multiclass(model_multiclass, state='eval')

Using Device: cuda


/tmp/ipykernel_2461889/1453464525.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/storage/coda1/p-cwiese7/0/yyu496/Model/checkpoint_m

Testing Metrics: Testing Loss: 0.1505080 || Testing Accuracy: tensor([0.9891, 1.0000, 0.9828, 1.0000], device='cuda:0')
                                       Testing Precision: tensor([1.0000, 0.9837, 1.0000, 1.0000], device='cuda:0') || Testing Recall: tensor([0.9891, 1.0000, 0.9828, 1.0000], device='cuda:0')
                                       Testing AUROC: tensor([0.9988, 0.9997, 1.0000, 1.0000], device='cuda:0')
